In [2]:
from narupa.app.client import NarupaClient
#from openmm_calculator import OpenMMCalculator
#from imd_openmm import openmm_ase_frame_server, start_imd
from narupa.ase.converter import frame_data_to_ase

import numpy as np
import subprocess
import nglview
import shlex


## Run the server

Run the server. Here, we're using ASE, but it could be LAMMPS or OpenMM or anything else. 

In [3]:
#imd = start_imd(input_xml)
#process = subprocess.Popen(['./lmp < data.peptide'], shell=True, cwd='/home/simon/narupa-protocol/python-libraries/narupa-lammps/examples',stdout=subprocess.PIPE)
#stdout = process.communicate()[0]
#print('STDOUT:{}'.format(stdout))

process = subprocess.Popen(['./lmp < data.peptide'], shell=True, cwd='/home/simon/narupa-protocol/python-libraries/narupa-lammps/examples',stdout=subprocess.PIPE)
i=1
while True:
    output = process.stdout.readline()
    #print("doing sothing", output)
    #print(output, len(output))
    #print(process.poll())
    if len(output) == 0 and process.poll() is not None:
        break
    if output:
        print(output.strip())
    i +=1
    if i >10:
        break
rc = process.poll()


Run dynamics in background thread

## Start an IMD client

In [4]:
class CustomClient(NarupaClient):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._view = None
    
    @property
    def view(self):
        if self._view is None:
            atoms = frame_data_to_ase(self.first_frame, topology=True, positions=False)
            atoms.set_positions(np.array(self.latest_frame.particle_positions) * 10)
            self._view = nglview.show_ase(atoms)
        return self._view
    
    def _on_frame_received(self, frame_index, frame):
        super()._on_frame_received(frame_index, frame)
        self.view.set_coordinates({0: np.array(self.latest_frame.particle_positions) * 10})

In [62]:
client = CustomClient(address='localhost',trajectory_port=8080, imd_port=8081)


In [63]:
atoms = frame_data_to_ase(client.first_frame, topology=True, positions=False)
atoms.set_positions(np.array(client.latest_frame.particle_positions) * 10)

In [64]:
client.view

NGLWidget()

In [65]:
from narupa.imd.particle_interaction import ParticleInteraction
interaction = ParticleInteraction("5", "0", position=(0,0,0), particles=(68), scale=0.015, interaction_type="spring")
interaction_id = client.start_interaction()
client.update_interaction(interaction_id, interaction)

In [66]:
client.stop_interaction(interaction_id)